In [1]:
import dml
from pymongo import MongoClient
import prov.model
import datetime
import json
import uuid
import folium
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [3]:
client = MongoClient()
db = client.repo
collection = db.jhs2018_rpm1995.kmeansdata
df = pd.DataFrame(list(collection.find()))

dir_path = "C:/Users/Jonathan"

latitude = [0]*len(df)
longitude = [0]*len(df)
for i in range(len(df)):
    latitude[i] = df['coordinates'][i][0]
    longitude[i] = df['coordinates'][i][1]
    

df['latitude'] = latitude
df['longitude']= longitude
#df['reallat'] = reallat
#df['reallong'] = reallong
df = df[(df.charge_count != 0)|(df.hubway_count != 0)|(df.open_count != 0)]

df[['charge_count','hubway_count','open_count','latitude','longitude']] = MinMaxScaler().fit_transform(df[['charge_count','hubway_count','open_count','latitude','longitude']])

In [4]:
df['latitude'] = df.latitude + 1.0
df['longitude'] = df.longitude + 1.0
df['charge_count'] = df.charge_count
df['hubway_count'] = df.hubway_count
df['open_count'] = df.open_count
df = df.reset_index()

In [8]:
#filenamemap = os.path.join(dir_path, "kmeans.html")
map_osm = folium.Map(location=[39, -98.1], zoom_start=4)
#marker_cluster = folium.MarkerCluster().add_to(map_osm)

y_pred = KMeans(n_clusters = 11, random_state = 0)
y_pred = y_pred.fit_predict(df[['charge_count','hubway_count','open_count','latitude','longitude']])
colors =             ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
             'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
             'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
             'gray', 'black', 'lightgray']
color = []
for clust in y_pred:
    color.append(colors[clust])
df['colors'] = color

for point in range(0, len(df)):
    folium.Marker(df['coordinates'][point], icon = folium.Icon(color = df['colors'][point])).add_to(map_osm)
    
map_osm